In [ ]:
import pandas as pd

In [ ]:
1. jewerly_place 데이터에 brand_name 컬럼을 추가

In [ ]:
# # 1) 두 파일 불러오기
# df_place = pd.read_csv("/Users/t2024-m0246/Downloads/sparta-project/reluxury/data/jewelry_place_word_with_category.csv")
# df_items = pd.read_csv("/Users/t2024-m0246/Downloads/sparta-project/reluxury/data/jewelry_items_word_with_category.csv")

# # 2) brand_id - brand_name 매핑 딕셔너리 생성
# brand_map = df_items[['brand_id', 'brand_name']].drop_duplicates()

# # 3) place 데이터에 brand_name 추가 (merge 활용)
# df_place_with_name = df_place.merge(brand_map, on="brand_id", how="left")

# # 4) 확인
# print(df_place_with_name.head())

# # 5) 새로운 CSV로 저장
# df_place_with_name.to_csv("jewelry_place_with_brandname.csv", index=False, encoding="utf-8-sig")


In [ ]:
# import pandas as pd

# # 1) CSV 불러오기   # items 파일에서 브랜드명 추가된 place 파일 사용
# df = pd.read_csv("/Users/t2024-m0246/Downloads/sparta-project/reluxury/data/jewelry_place_with_brandname.csv")

# # 2) 분석 대상 브랜드 리스트
# target_brands = ["불가리", "까르띠에", "반클리프", "티파니앤코"]

# # 3) 필터링된 분석용 데이터프레임 생성
# df_selected = df[df["brand_name"].isin(target_brands)]

# # 4) 확인
# print(df_selected.head())
# print("총 데이터 수:", len(df_selected))


퍼플렉시티 설치

In [ ]:
pip install pplx

In [ ]:
pplx_client.py 생성 

In [ ]:
pip install -U openai python-dotenv pandas

반클리프, 까르띠에, 불가리, 티파니만 필터링해서 질문했더니

In [ ]:

# 1회성 테스트 버전. 

import os
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

# 1) 환경 변수 로드 (.env 안에 PPLX_API_KEY 있어야 함)
load_dotenv()
api_key = os.getenv("PPLX_API_KEY")
if not api_key:
    raise ValueError("환경변수 PPLX_API_KEY가 없습니다. .env 파일 확인!")

# 2) Perplexity(OpenAI 호환) Client 초기화
client = OpenAI(
    api_key=api_key,
    base_url="https://api.perplexity.ai"  # ★ 중요
)

# 3) 데이터 불러오기 (브랜드 필터링된 CSV)
df = pd.read_csv("/Users/t2024-m0246/Downloads/sparta-project/reluxury/data/jewelry_place_with_brandname.csv")
target_brands = ["불가리", "까르띠에", "반클리프", "티파니앤코"]
df_selected = df[df["brand_name"].isin(target_brands)].copy()

# 4) 질문 → 관련 행 일부 추려서 LLM에 전달
def ask_places(question: str, df_context: pd.DataFrame = None, max_rows: int = 30):
    if df_context is None:
        df_context = df_selected

    # 테이블을 CSV 문자열로 변환 (앞부분만)
    cols = [c for c in ["brand_name", "place_name", "location_lv1", "location_lv2", "prelative_word"] if c in df_context.columns]
    table_str = df_context[cols].head(max_rows).to_csv(index=False)

    # LLM 질의 (OpenAI SDK 인터페이스)
    response = client.chat.completions.create(
        model="sonar",
        messages=[
            {"role": "system", "content": "너는 명품 주얼리 매장/브랜드 전문가야. 제공된 테이블 데이터만 근거로 한국어로 답해."},
            {"role": "user", "content": f"[데이터]\n{table_str}\n\n[질문]\n{question}"}
        ]
    )
    return response.choices[0].message.content

# 5) 사용 예시
print("df_selected shape:", df_selected.shape)
print(df_selected.head(2))

q = "청담에 반클리프 매장이 있나요? 매장명과 지역 알려줘."
answer = ask_places(q)
print("\n[Answer]\n", answer)


청담에 반클리프 매장이 왜 없다는겨 
-> 파일 열어보면 기타(청담 메종) 로 구분되어있음. 

In [ ]:
# 유지보수 용이한 버전으로 재작성

import os
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

# ------------------------
# ① 환경설정 & 클라이언트 준비
# ------------------------
load_dotenv()
api_key = os.getenv("PPLX_API_KEY")
if not api_key:
    raise ValueError("환경변수 PPLX_API_KEY가 없습니다. .env 파일 확인!")

client = OpenAI(api_key=api_key, base_url="https://api.perplexity.ai")

# ------------------------
# ② 데이터 준비
# ------------------------
df = pd.read_csv("/Users/t2024-m0246/Downloads/sparta-project/reluxury/data/jewelry_place_with_brandname.csv")
target_brands = ["불가리", "까르띠에", "반클리프", "티파니앤코"]
df_selected = df[df["brand_name"].isin(target_brands)].copy()

# ------------------------
# ③ 질문 함수 정의
# ------------------------
def ask_places(question: str, df_context: pd.DataFrame = df_selected, max_rows: int = 30) -> str:
    """자연어 질문 → DF 일부를 붙여 LLM 응답"""
    cols = [c for c in ["brand_name", "place_name", "location_lv1", "location_lv2", "prelative_word"] if c in df_context.columns]
    table_str = df_context[cols].head(max_rows).to_csv(index=False)

    response = client.chat.completions.create(
        model="sonar",
        messages=[
            {"role": "system", "content": "너는 명품 주얼리 매장/브랜드 전문가야. 제공된 테이블 데이터만 근거로 한국어로 답해."},
            {"role": "user", "content": f"[데이터]\n{table_str}\n\n[질문]\n{question}"}
        ]
    )
    return response.choices[0].message.content

# ------------------------
# ④ 사용 예시
# ------------------------
if __name__ == "__main__":
    print("데이터 크기:", df_selected.shape)

    # 자유롭게 질문하기
    questions = [
        "불가리 매장은 어느 지역에 분포해 있나요?",
        "티파니앤코 매장 중 서울에 있는 곳 알려줘.",
        "강남에서 까르띠에 매장이 있나요?"
    ]

    for q in questions:
        print("\n질문:", q)
        print("답변:", ask_places(q))


In [ ]:

df_clean = pd.read_csv("/Users/t2024-m0246/Downloads/sparta-project/reluxury/data/cafe_posts_clean.csv")


In [ ]:
df_clean.head()

In [ ]:
df_clean.columns

In [ ]:
# clean + place + sentiment 파일 기반 LLM 질문 - 유지보수 용이한 버전 

import os
import re
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

# ------------------------
# ① 환경설정 & 클라이언트 준비
# ------------------------
load_dotenv()
api_key = os.getenv("PPLX_API_KEY")
if not api_key:
    raise ValueError("환경변수 PPLX_API_KEY가 없습니다. .env 파일 확인!")

client = OpenAI(api_key=api_key, base_url="https://api.perplexity.ai")

# ------------------------
# ② 데이터 불러오기
# ------------------------
BASE_PATH = "/Users/t2024-m0246/Downloads/sparta-project/reluxury/data"

df_clean = pd.read_csv(f"{BASE_PATH}/cafe_posts_clean.csv")
df_place = pd.read_csv(f"{BASE_PATH}/jewelry_place_with_brandname.csv")
df_sent = pd.read_csv(f"{BASE_PATH}/place_sentiment_keywords.csv")

# ------------------------
# ③ 관련 데이터 subset 추출 함수
# ------------------------
def keyword_subset(df, query: str, max_rows: int = 10):
    """간단 키워드 검색으로 Clean subset 추출"""
    toks = re.findall(r"[A-Za-z0-9가-힣]{2,}", query)
    toks = [t.lower() for t in toks if len(t) > 1]

    def row_hit(row):
        hay = f"{row.get('title','')} {row.get('text','')}".lower()
        return any(tok in hay for tok in toks)

    sub = df[df.apply(row_hit, axis=1)]
    if len(sub) == 0:
        return df.head(max_rows).copy()
    return sub.head(max_rows).copy()

# ------------------------
# ④ LLM 질문 함수
# ------------------------
def ask_clean(question: str, max_rows: int = 10):
    # 1) Clean subset
    clean_sub = keyword_subset(df_clean, question, max_rows=max_rows)
    clean_str = clean_sub[["title","text"]].to_csv(index=False)

    # 2) Place (브랜드/매장 메타)
    place_str = df_place[["brand_name","place_name","location_lv1","location_lv2"]].dropna().head(10).to_csv(index=False)

    # 3) Sentiment (사전 키워드)
    sent_str = df_sent.head(10).to_csv(index=False)

    # 4) LLM 호출
    response = client.chat.completions.create(
        model="sonar",
        messages=[
            {"role": "system", "content": "너는 명품 주얼리 커뮤니티 데이터 분석 전문가야. Clean 데이터(게시글) 내용을 중심으로 답하되, Place와 Sentiment는 참고자료로만 활용해."},
            {"role": "user", "content": f"[Clean 데이터]\n{clean_str}\n\n[Place 사전정보]\n{place_str}\n\n[Sentiment 사전]\n{sent_str}\n\n[질문]\n{question}"}
        ]
    )
    return response.choices[0].message.content

# ------------------------
# ⑤ 사용 예시
# ------------------------
if __name__ == "__main__":
    print("df_clean shape:", df_clean.shape)
    print("df_place shape:", df_place.shape)
    print("df_sent shape:", df_sent.shape)

    q1 = "불가리 세르펜티에 대한 소비자 반응은 어떤가요?"
    q2 = "청담 지역에서 가장 많이 언급된 브랜드는 무엇인가요?"

    for q in [q1, q2]:
        print("\n질문:", q)
        print("답변:", ask_clean(q))

# ------------------------
# 답변이 너무 길어


In [ ]:
# ------------------------
# ⑤ 사용 예시
# ------------------------
if __name__ == "__main__":
    print("df_clean shape:", df_clean.shape)
    print("df_place shape:", df_place.shape)
    print("df_sent shape:", df_sent.shape)

    q1 = "까르띠에 탱크머스트에 대한 소비자 반응은 어떤가요? 3줄 요약해주세요"
    q2 = "경기 지역에서 가장 많이 언급된 브랜드는 무엇인가요? 상위 3개 브랜드만 알려주세요"

    for q in [q1, q2]:
        print("\n질문:", q)
        print("답변:", ask_clean(q))
        
# ------------------------
# 답변의 정확도를 확인해보고 싶어

해당 답변 내용이 정확한지 어떻게 알 수 있지? 
내가 확인해 볼 수 있는 내용으로 질문해보자.

In [ ]:
# ------------------------
# ⑤ 사용 예시
# ------------------------
if __name__ == "__main__":
    print("df_clean shape:", df_clean.shape)
    print("df_place shape:", df_place.shape)
    print("df_sent shape:", df_sent.shape)

    q1 = "게시판에서 댓글 수가 가장 많은 글 3개는 무엇인가요? 게시글번호와 제목, 댓글 수를 알려주세요"
    q2 = "2024년 3월,4월,5월에 작성된 글 중에서 가장 많이 언급된 브랜드 3가지와 언급 횟수를 알려주세요"

    for q in [q1, q2]:
        print("\n질문:", q)
        print("답변:", ask_clean(q))
        
# ------------------------
# 게시글 번호 정보가 왜 없다는겨 

In [ ]:
# ------------------------
# ⑤ 사용 예시
# ------------------------
if __name__ == "__main__":
    print("df_clean shape:", df_clean.shape)
    print("df_place shape:", df_place.shape)
    print("df_sent shape:", df_sent.shape)

    q1 = "서울에서 만족도가 가장 높은 매장 3곳은 어디인가요? 매장명과 지역, 만족도 정도를 알려주세요"
    q2 = "서울에서 만족도가 가장 낮은 매장 3곳은 어디인가요? 매장명과 지역, 만족도 정도를 알려주세요"

    for q in [q1, q2]:
        print("\n질문:", q)
        print("답변:", ask_clean(q))